In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import os

In [2]:
# Project Root Path
projRoot = Path.cwd().parents[0]
path_rawdata = projRoot.joinpath('data/raw')

In [11]:
# Filepaths to raw data
filepaths = sorted(path_rawdata.rglob('[教學]*.csv'))

In [12]:
'''
# Stage1: '...' and '無' into np.na

# creating stage1 folder
path_stage1 = projRoot.joinpath('data/stage1')
path_stage1.mkdir(exist_ok=True)

for file in filepaths:
    df = pd.read_csv(file, na_values=['...', '無'])
    csv_filename = f'{path_stage1}/stage1_{file.name}'
    df.to_csv(csv_filename, index=False)
'''

/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_3818/2392185783.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=['...', '無'])


In [3]:
# Filepaths to stage 1 data
path_stage1 = projRoot.joinpath('data/stage1')
filepaths_s1 = sorted(path_stage1.rglob('[教學]*.csv'))

In [7]:
# Stage2: Aggregate to school-year data (on school level)

## creating stage2 folder
path_stage2 = projRoot.joinpath('data/stage2')
path_stage2.mkdir(exist_ok=True)

## 2-1 long to wide on '學期' (by '學年度' and '學校統計處代碼')
for file in filepaths_s1:
    df = pd.read_csv(file)
    df[['學年度', '學校統計處代碼']] = df[['學年度', '學校統計處代碼']].astype('object')
    cols_to_drop = [col for col in df.columns if ('%' in col) or ('平均' in col)] #平均、百分比等資訊等aggregate之後再重新計算
    df = df.drop(cols_to_drop, axis=1)
    print(cols_to_drop)
    if ('休學' in f'{file}') or ('退學' in f'{file}'): #目前休退學人數統計表各自「以「系(所)」」為單位的表，人數加總不一致，所以刪掉
        df = df.drop(['在學學生數'], axis=1)

    numeric_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
    numeric_cols = list(filter(lambda x: x not in ['學期', '系所代碼'], numeric_cols))
    if ('學期' in list(df.columns)) and (numeric_cols != []): #有分1、2學期的資料表，依學期做long 轉wide
        df = df.groupby(['學年度', '學校統計處代碼', '學期'])[numeric_cols].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
        df.reset_index(inplace=True)
        df = df.pivot(index=['學年度', '學校統計處代碼'], columns= '學期', values= numeric_cols)
        df.columns = [f"{col[0]}_{col[1]}" for col in df.columns]
        df.reset_index(inplace=True)
    elif numeric_cols == []: #如果整張表都沒有數值變項，就先不採用（跟指標較無關，或再想辦法呈現）
        filepaths_s1.remove(file)
        print('not in use: '+ f'{file.name}')
        continue
    elif '學年底' in f'{file}': #「以學年底狀態統計」的表，視作是第2學期的數據，變項名稱加上後綴'_2'
        df = df.rename(columns={col: col+'_2' for col in df[numeric_cols]})

    numeric_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
    numeric_cols = list(filter(lambda x: x not in ['學期', '系所代碼'], numeric_cols))
    # aggregate to school level
    df = df.groupby(['學年度', '學校統計處代碼'])[numeric_cols].agg(lambda x: np.nan if np.all(pd.isna(x)) else x.sum())
    df.reset_index(inplace=True)
    csv_filename = f'{path_stage2}/{file.name}'
    df.to_csv(csv_filename, index=False)

[]
[]
[]
[]


/var/folders/ty/15vk89y91j50b13mgsdgzxc80000gn/T/ipykernel_13428/2547165806.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


['當學年度新生註冊率(%)D=〔(C+E)/(A-B+E)〕＊100％']
['當學年度各學制新生註冊率(%)E=〔(C+D)/(A-B+D)〕*100％']
[]
[]
[]
[]
[]
[]
['前一學年度學生就學穩定率(%)(C)(C=B/A)']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['境外學位生數之在學比率(%)']
['外國學生數之在學比率(%)']
['僑生及港澳生數之在學比率(%)']
['大陸地區來臺學位生數之在學比率(%)']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['編制外專任教師比率(%)']
[]
[]
[]
[]
[]
not in use: 教18.私立學校積欠教師薪資情形-以「校」統計.csv
[]
[]
[]
[]
[]
[]
[]
[]
['專任教師平均每週授課時數-教授', '專任教師平均每週授課時數-副教授', '專任教師平均每週授課時數-助理教授', '專任教師平均每週授課時數-講師', '專任教師平均每週授課時數-其他教師']
not in use: 教4-2.專任教師每週授課時數-以「校」統計.csv
['專任教師增減比率(%) 111學年度', '專任教師增減比率(%) 110學年度', '專任教師增減比率(%) 109學年度']
['專任教師通過升等比率(%)(F)=(E/A)*100%']
[]


In [10]:
# Filepaths to stage 2 data
filepaths_s2 = sorted(path_stage2.rglob('[教學]*.csv'))

In [13]:
# Stage3: All stage2 datasets merged
dfs = {}
for file in filepaths_s2:
    dfs[file.name] = pd.read_csv(file)
    dfs[file.name][['學年度', '學校統計處代碼']] = dfs[file.name][['學年度', '學校統計處代碼']].astype('string') #merge的時候，校統計處代碼要先統一長度
    dfs[file.name]['學校統計處代碼'] = dfs[file.name]['學校統計處代碼'].str.zfill(4)
    dfs[file.name]['學年度'] = dfs[file.name]['學年度'].str.zfill(3)
# merge all
keys = list(dfs.keys())
merged_df = dfs[keys[0]]
for i in range(1, len(keys)):
    merged_df = pd.merge(merged_df, dfs[keys[i]], how='outer')

## creating stage3 folder
path_stage3 = projRoot.joinpath('data/stage3')
path_stage3.mkdir(exist_ok=True)

csv_filename = f'{path_stage3}/學教_merged.csv'
merged_df.to_csv(csv_filename, index=False)

